In [1]:
import torch
import gpytorch as gpt
import botorch
from botorch.models import SingleTaskMultiFidelityGP, SingleTaskGP
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize
from botorch.fit import fit_gpytorch_mll, fit_gpytorch_mll_torch
from torch.optim import Adam
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import matplotlib.pyplot as plt
import HeBz
import pickle
from scipy.spatial.distance import cdist

# Use CPU for this example
device = torch.device("cuda")
dtype = torch.float64

In [2]:
zpoints = np.linspace(0,7,100)
xpoints = np.linspace(0,5,100)
data_eval = []
for i in range(100):
    data_eval.append([4.5,0,zpoints[i],1.0])
data_eval = np.array(data_eval)
data_eval = torch.from_numpy(data_eval)

In [3]:
#Load the data from file
V = np.load('../Final/HeBz_sector_2475.npy')
x = np.array([])
y = np.array([])
z = np.array([])
data_points = []
Pot = np.array([])
Pot_diff = np.array([])
for array in V:
    x = np.append(x,array[0])
    y = np.append(y,array[1])
    z = np.append(z,array[2])
    data_points.append([array[0],array[1],array[2],1.0])
    Pot = np.append(Pot,array[3])
V2 = np.load('../Final/new_pts_70.npy')
for array in V2:
    x = np.append(x,array[0])
    y = np.append(y,array[1])
    z = np.append(z,array[2])
    data_points.append([array[0],array[1],array[2],1.0])
    Pot = np.append(Pot,array[3])
data_points = np.array(data_points)
noise = 1e-6*np.ones_like(Pot)

In [4]:
# -----------------------------
# 1. Generate Source Task Data
# -----------------------------
DFTdata0 = np.load("pbe0_113850_CP_D4_processed.npy")
ind = np.where(np.isclose(DFTdata0[:,2],0.09459459))
refdata = DFTdata0[ind]
refdata[:,2] = -1*refdata[:,2]
DFTdata = np.concatenate((DFTdata0,refdata))
ind = np.argsort(DFTdata[:,2], kind='stable')
DFTdatazsrt = DFTdata[ind]
ind = np.argsort(DFTdatazsrt[:,1], kind='stable')
DFTdatayzsrt = DFTdatazsrt[ind]
DFTdataxrem = DFTdatayzsrt[::8]
ind = np.argsort(DFTdataxrem[:,0], kind='stable')
DFTdataxremzsrt = DFTdataxrem[ind]
DFTdatazxrem = DFTdataxremzsrt
print(DFTdatazxrem)
DFTdata2 = np.load("pbe0_corr_sutirtha_CP_D4_full_processed.npy")
#print(DFTdatazxrem)
#print(np.min(DFTdata2[:,-1]))
DFTdatanew = np.concatenate((DFTdata2[:,0],DFTdata2[:,1],DFTdata2[:,2]),)
DFTtotal = np.concatenate((DFTdatazxrem[:,0:3],DFTdata2[:,0:3]))
data_source = [[x, y, z, w] for x, y, z, w in zip(DFTtotal[:,0],DFTtotal[:,1],DFTtotal[:,2],np.zeros(len(DFTtotal)))]
m_source = np.concatenate((DFTdatazxrem[:,-1],DFTdata2[:,-1]))
noise_source = 1e-6*np.ones_like(m_source)

[[ 0.00000000e+00  0.00000000e+00  9.45945946e-02 ...  1.21158616e-08
  -1.83603042e+04  1.00634447e+05]
 [ 0.00000000e+00  0.00000000e+00  8.51351351e-01 ... -2.11424265e-09
  -8.68586691e+04  4.83971583e+04]
 [ 0.00000000e+00  0.00000000e+00  1.60810811e+00 ...  3.99535650e-10
  -2.31209156e+04  7.70900596e+03]
 ...
 [ 7.00000000e+00  0.00000000e+00  5.39189189e+00 ...  1.37187109e-11
   4.48182311e-01  2.05744395e+00]
 [ 7.00000000e+00  0.00000000e+00  6.14864865e+00 ... -3.75382973e-11
  -2.01439312e-01  2.36286681e+00]
 [ 7.00000000e+00  0.00000000e+00  6.90540541e+00 ...  9.38433445e-13
   9.39158574e-02  2.36453566e+00]]


In [5]:
train_X = np.concatenate((data_points,data_source))
train_Y = np.concatenate((Pot,m_source))
train_Y = train_Y.reshape(-1,1)
train_Yvar = np.concatenate((noise,noise_source))
train_Yvar = train_Yvar.reshape(-1,1)
print(train_X.shape)

(19506, 4)


In [6]:
train_Y_trunc = train_Y[train_Y[:, 0]<=1000]
train_Yvar_trunc = train_Yvar[train_Y[:,0]<=1000]
train_X_trunc = train_X[train_Y[:, 0]<=1000, :]
print(train_X_trunc.shape)

(18162, 4)


In [62]:
state_dict = torch.load('isaac/my_model_multfidDFT16kwNoiseRefBetter.pth')
model = SingleTaskMultiFidelityGP(torch.from_numpy(train_X_trunc), torch.from_numpy(train_Y_trunc), torch.from_numpy(train_Yvar_trunc), data_fidelities=[len(train_X_trunc[0]) - 1], input_transform=Normalize(d=4),outcome_transform=Standardize(m=1))
mll = ExactMarginalLogLikelihood(model.likelihood, model)
model.load_state_dict(state_dict)


tensor([[8.6422]], dtype=torch.float64)


In [8]:
def matern_kernel_np(x1, x2, lengthscale):
    """
    Compute Matern kernel between x1 and x2 with given lengthscale and smoothness nu.
    """
    # pairwise distance
    r = cdist(x1 / lengthscale, x2 / lengthscale, metric='euclidean')
    #print(r)
    sqrt5 = np.sqrt(5.0)
    k = (1 + sqrt5 * r + 5 * r**2 / 3) * np.exp(-sqrt5 * r)
    return k
def kernel_np(X1,X2,theta1,theta2,P,scale):
    # split x and z
    #print(X1,X2)
    x1, z1 = X1[..., :-1], X1[..., -1:]
    x2, z2 = X2[..., :-1], X2[..., -1:]
    x11_ = z1
    x21t_ = z2
    x21t_ = np.transpose(x21t_)
    #print(x11_,x21t_)
    cross_term_1 = (1 - x11_) * (1 - x21t_)
    bias_factor = cross_term_1 * np.power((1 + x11_ * x21t_),P)
    #print(bias_factor)
    return scale*(matern_kernel_np(x1,x2,lengthscale=theta1) + bias_factor*matern_kernel_np(x1,x2,lengthscale=theta2))

In [48]:
print(model.state_dict())

OrderedDict({'mean_module.raw_constant': tensor(22.0553, dtype=torch.float64), 'covar_module.raw_outputscale': tensor(814.6966, dtype=torch.float64), 'covar_module.base_kernel.kernels.0.raw_power': tensor([-0.1112], dtype=torch.float64), 'covar_module.base_kernel.kernels.0.raw_power_constraint.lower_bound': tensor(0., dtype=torch.float64), 'covar_module.base_kernel.kernels.0.raw_power_constraint.upper_bound': tensor(inf, dtype=torch.float64), 'covar_module.base_kernel.kernels.0.power_prior.concentration': tensor(3., dtype=torch.float64), 'covar_module.base_kernel.kernels.0.power_prior.rate': tensor(3., dtype=torch.float64), 'covar_module.base_kernel.kernels.0.covar_module_unbiased.raw_lengthscale': tensor([[0.1785, 2.0518, 0.1523]], dtype=torch.float64), 'covar_module.base_kernel.kernels.0.covar_module_unbiased.lengthscale_prior.concentration': tensor(3., dtype=torch.float64), 'covar_module.base_kernel.kernels.0.covar_module_unbiased.lengthscale_prior.rate': tensor(6., dtype=torch.floa

In [65]:
ell1 = model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale
ell2 = model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale
p = model.covar_module.base_kernel.kernels[0].power
oscale = model.covar_module.outputscale
learned_mean = model.mean_module.constant
train_x_trunc = torch.from_numpy(train_X_trunc)
train_y_trunc = torch.from_numpy(train_Y_trunc)
train_x_nrm = (train_x_trunc - torch.min(train_x_trunc,0)[0])/(torch.max(train_x_trunc,0)[0] -torch.min(train_x_trunc,0)[0])

print(train_x_nrm)
covar_matrix = model.covar_module(train_x_nrm,train_x_nrm).to_dense()
covar_matrix = covar_matrix + model.likelihood.noise* torch.eye(covar_matrix.size(0))
inverse_cov = torch.linalg.inv(covar_matrix)
stdvs = train_y_trunc.std(dim=-2, keepdim=True)
means = train_y_trunc.mean(dim=-2, keepdim=True)
train_y_std = (train_y_trunc - means)/stdvs
prod = torch.matmul(inverse_cov,train_y_std - learned_mean).to_dense().detach().numpy()   #Store this
print(prod)
prod2 = torch.linalg.solve(covar_matrix, train_y_std - learned_mean)
#Store all parameters in a dictionary
my_dict = {
    "outputscale": model.covar_module.outputscale.detach().numpy(),
    "theta1": model.covar_module.base_kernel.kernels[0].covar_module_unbiased.lengthscale.detach().numpy(),
    "theta2":model.covar_module.base_kernel.kernels[0].covar_module_biased.lengthscale.detach().numpy(),
    "learned_mean": model.mean_module.constant.detach().numpy(),
    "power": model.covar_module.base_kernel.kernels[0].power.detach().numpy(),
    "data":train_x_trunc.numpy(),
    "product": prod,
    "stddevy": stdvs.detach().numpy(),
    "meany": means.detach().numpy()
    
}
with open("data2.pkl", "wb") as f:
    pickle.dump(my_dict, f)

tensor([[0.6599, 0.5467, 0.5941, 1.0000],
        [0.4949, 0.5714, 0.5941, 1.0000],
        [0.4968, 0.2662, 0.5941, 1.0000],
        ...,
        [0.5939, 0.4920, 0.8647, 0.0000],
        [0.5143, 0.0000, 0.8647, 0.0000],
        [0.6429, 0.0000, 0.0133, 0.0000]], dtype=torch.float64)
[[   9.19936053]
 [  29.38108591]
 [ -61.91943051]
 ...
 [   5.82816816]
 [ -34.95164383]
 [-200.36503508]]


In [44]:
with torch.no_grad():
    posterior_HF = model.posterior(torch.tensor([[4.5, 0., 6.718, 1.]]))
    mean_HF = posterior_HF.mean.cpu().detach().numpy()
    lower_HF, upper_HF = posterior_HF.confidence_region()

In [66]:
#Manual prediction
with open('data2.pkl', 'rb') as file:
    gp_data = pickle.load(file)
#print(gp_data)
predval = []
newv = []
tx = gp_data["data"]
print(np.min(tx,0))
print(np.max(tx,0))
tx_nrm = (tx - np.min(tx,0))/(np.max(tx,0) - np.min(tx,0))
print(data_eval.numpy()[-5])
for new in [[4.5, 0, 6.718, 1.]]:
    new_nrm = (new - np.min(tx,0))/(np.max(tx,0) - np.min(tx,0))
    new_nrm = new_nrm[np.newaxis,...]
    print(new_nrm)
    k = np.transpose(kernel_np(tx_nrm,new_nrm,gp_data['theta1'],gp_data['theta2'],gp_data['power'],gp_data['outputscale']))
    k_ =  torch.transpose(model.covar_module(torch.from_numpy(tx_nrm),torch.from_numpy(new_nrm)).to_dense(),0,1)
    new_val = learned_mean + k_ @ prod2
    #print(gp_data["product"],k)
    evalz = gp_data['learned_mean'] + np.dot(k,gp_data["product"])
    print(new_val*104, evalz*104.9113)
    newv.append(gp_data['meany'] + gp_data['stddevy']*new_val.detach().numpy())
    predval.append(gp_data['meany'] + gp_data['stddevy']*evalz)
predval = np.array(predval).flatten()

print(predval)
print(newv)
print(mean_HF.flatten())

[ 0.          0.         -0.09459459  0.        ]
[7.  3.5 7.  1. ]
[4.5        0.         6.71717172 1.        ]
[[0.64285714 0.         0.96025143 1.        ]]
tensor([[-9.4027]], dtype=torch.float64, grad_fn=<MulBackward0>) [[-8.83324499]]
[-0.19109097]
[array([[-0.84297114]])]
[-0.84304046]
